In [8]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import time
import numpy as np
import pandas as pd
s3 = boto3.resource('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

BUCKET      = sagemaker_session.default_bucket()
TRAINING_FILE     = 'train_prediction.py'
INFERENCE_FILE = 'inference.py'
SOURCE_DIR = 'source_dir'

DATA_PREFIX            = 'RF_SCIKIT'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

ENDPOINT_NAME = 'model-random-forest-new'

MODEL_NAME = ENDPOINT_NAME

In [9]:
from sklearn.model_selection import train_test_split

SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]
import boto3
def split():
    bucket_name = 'sagemaker-data-icr'
    s3 = boto3.resource('s3')
    data_key = 'first phenomes and genomes.csv'
    data_location = 's3://{}/{}'.format(bucket_name, data_key)
    df = pd.read_csv(data_location)
    #data = df.drop(['Unnamed: 0'],axis=1)
    model_data = df.fillna(df.mean())
    COLUMNS = list(df.columns)
    
#     #i = str(input("enter a feature name "))
#     X =df1.drop([loc], axis = 1)
   
#     y =df1[loc]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
#     #X_train_col = X_train.columns
#     return X_train, X_test, y_train, y_test
    
    i = str(input("enter a feature name "))
    #for i in col:
   
    # split data into train and test sets
    seed      = SEED
    val_size  = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]
    
    _train, _val, _test = np.split(model_data.sample(frac=1, random_state=123), 
                                                  [int(0.7 * len(model_data)), int(0.9*len(model_data))]) 
    #train =  train_data.drop([i], axis=1)#, index=False)
    #val =   validation_data.drop([i], axis=1)#, index=False)
    #test =  test_data.drop([i], axis=1)#, index=False)

# Dropping the target value, as we will use this CSV file for batch transform
    #test = test_data.drop([i], axis=1).to_csv('test.csv', index=False, header=False)

    return _train, _val, _test ,COLUMNS


In [10]:
import os
def save_data_locally(location, train, val, test,COLUMNS):

#     header = ','.join(COLUMNS)
#     print(header)
#     print(train)
    os.makedirs(f'data/{location}/train')
    np.savetxt(f'data/{location}/train/{location}_train.csv', train, header=','.join(COLUMNS), delimiter=',', fmt='%.10f',comments="")
    
    os.makedirs(f'data/{location}/val')
    np.savetxt(f'data/{location}/val/{location}_val.csv',     val,  header = ','.join(COLUMNS), delimiter=',', fmt='%.10f',comments="")
    
    os.makedirs(f'data/{location}/test')
    np.savetxt(f'data/{location}/test/{location}_test.csv',   test,  header = ','.join(COLUMNS),  delimiter=',', fmt='%.10f',comments="")


In [18]:
from sagemaker.sklearn.estimator import SKLearn

def launch_training_job(location):
    # clear out old versions of the data
    s3_bucket = s3.Bucket(BUCKET)
    full_input_prefix = f'{DATA_PREFIX}/model_prep/{location}'
    s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    local_folder = f'data/{location}'
    inputs = sagemaker_session.upload_data(path=local_folder, 
                                            key_prefix=full_input_prefix)
    
    print(f'Training data uploaded: {inputs}')
    
    _job = 'gp-{}'.format(location.replace('_', '-'))
    full_output_prefix = f'{DATA_PREFIX}/model_artifacts/{location}'
    s3_output_path = f's3://{BUCKET}/{full_output_prefix}'
    
    code_location = f's3://{BUCKET}/{full_input_prefix}/code'
    # Return the estimator object
    return s3_output_path,_job, code_location ,inputs,location 

from sagemaker.sklearn.estimator import SKLearn

LOCATIONS  = ['100SDW_EIAR_2009_RF','100SDW_EIAR_2008_RF','100SDW_EU_2008_RF','100SDW_EU_2009_RF']
PARALLEL_TRAINING_JOBS =2

import shutil
import os

estimators = []

shutil.rmtree('data', ignore_errors=True)
for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    _train, _val, _test ,COLUMNS = split()
    save_data_locally(loc, _train, _val, _test ,COLUMNS)
    
    s3_output_path,_job, code_location,inputs,location = launch_training_job(loc)
    _estimator = SKLearn(
        entry_point=TRAINING_FILE, # script to use for training job
        role=role,
        source_dir=SOURCE_DIR, # Location of scripts
        train_instance_count=1,
        train_instance_type=TRAIN_INSTANCE_TYPE,
        framework_version='0.23-1',# 0.23-1 is the latest version
        output_path=s3_output_path,# Where to store model artifacts
        base_job_name=_job,
        code_location=code_location,
        hyperparameters =  {'model-name': location})
    
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    
    
    train_input = sagemaker.s3_input(s3_data=inputs+'/train', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    val_input   = sagemaker.s3_input(s3_data=inputs+'/val', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    
    remote_inputs = {'train': train_input, 'validation': val_input}
  
    
    #eval_set=[(train_input2, val_input2)]
    #eval_set = {'train': train_input2,'test': val_input2}


    _estimator.fit(remote_inputs,wait = False)#,eval_set)
    estimators.append(_estimator)

    
    print('{} training jobs launched: {}'.format(len(estimators), estimators))
    

enter a feature name 100SDW_EIAR_2009_RF


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training data uploaded: s3://sagemaker-ap-south-1-580246529711/RF_SCIKIT/model_prep/100SDW_EIAR_2009_RF
1 training jobs launched: [<sagemaker.sklearn.estimator.SKLearn object at 0x7f010ccfae80>]
enter a feature name 100SDW_EU_2008_RF


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training data uploaded: s3://sagemaker-ap-south-1-580246529711/RF_SCIKIT/model_prep/100SDW_EIAR_2008_RF
2 training jobs launched: [<sagemaker.sklearn.estimator.SKLearn object at 0x7f010ccfae80>, <sagemaker.sklearn.estimator.SKLearn object at 0x7f010cbf9f98>]


In [19]:
#APB_PAT_2001_RF2
#[100SDW_EIAR_2009_RF','100SDW_EU_2008_RF']

#2014DSPHY-1nd-layer
print()
print(f'{len(estimators)} training jobs launched: {[x.latest_training_job.name for x in estimators]}')


2 training jobs launched: ['gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965', 'gp-100SDW-EIAR-2008-RF-2020-07-22-10-56-28-902']


In [20]:
def wait_for_training_job_to_complete(estimator):
    job = estimator.latest_training_job.job_name
    print(f'Waiting for job: {job}')
    status = estimator.latest_training_job.describe()['TrainingJobStatus']
    while status == 'InProgress':
        time.sleep(45)
        status = estimator.latest_training_job.describe()['TrainingJobStatus']
        if status == 'InProgress':
            print(f'{job} job status: {status}')
    print(f'DONE. Status for {job} is {status}\n')
# wait for the jobs to finish
for est in estimators:
    wait_for_training_job_to_complete(est)


Waiting for job: gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965
gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 job status: InProgress
gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 job status: InProgress
gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 job status: InProgress
gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 job status: InProgress
gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 job status: InProgress
DONE. Status for gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965 is Completed

Waiting for job: gp-100SDW-EIAR-2008-RF-2020-07-22-10-56-28-902
DONE. Status for gp-100SDW-EIAR-2008-RF-2020-07-22-10-56-28-902 is Completed



In [21]:
from sagemaker.sklearn.estimator import SKLearn
job_name='gp-100SDW-EIAR-2009-RF-2020-07-22-10-56-19-965'
my_estimator = SKLearn.attach(job_name)
my_estimator

#aws sagemaker describe-training-job --training-job-name 'GENOME-PHENOME-2020-05-27-13-03-03'

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


2020-07-22 11:00:47 Starting - Preparing the instances for training
2020-07-22 11:00:47 Downloading - Downloading input data
2020-07-22 11:00:47 Training - Training image download completed. Training in progress.
2020-07-22 11:00:47 Uploading - Uploading generated training model
2020-07-22 11:00:47 Completed - Training job completed2020-07-22 10:59:45,564 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-07-22 10:59:45,566 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-22 10:59:45,578 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-22 10:59:45,968 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-22 10:59:45,981 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-22 10:59:45,993 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-07-22 10:59:46,0

In [22]:
estimator = estimators[0]
# inference.py is the entry_point for when we deploy the model
# Note how we do NOT specify source_dir again, this information is inherited from the estimator
model = estimator.create_model(role=role, entry_point='inference.py')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [23]:
from sagemaker.multidatamodel import MultiDataModel

# This is where our MME will read models from on S3.
model_data_prefix = f's3://{BUCKET}/{DATA_PREFIX}/{MULTI_MODEL_ARTIFACTS}/'

gp = MultiDataModel(name=MODEL_NAME,
                     model_data_prefix=model_data_prefix,
                     model=model,# passing our model
                     sagemaker_session=sagemaker_session)

In [25]:
predictor = gp.deploy(initial_instance_count=1,
                       instance_type=ENDPOINT_INSTANCE_TYPE,
                       endpoint_name=ENDPOINT_NAME)

---------------!

In [27]:
# No models visible!
list(gp.list_models())

[]

In [29]:
for est in estimators:
    artifact_path = est.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
    model_name = artifact_path.split('/')[-4]+'.tar.gz'
    # This is copying over the model artifact to the S3 location for the MME.
    gp.add_model(model_data_source=artifact_path, model_data_path=model_name)

In [31]:
list(gp.list_models())


['100SDW_EIAR_2009_RF.tar.gz']

In [33]:

start_time = time.time()
out_arr = np.random.randint(low = 0, high = 1099, size = 4799)

predicted_value = predictor.predict(out_arr, target_model='100SDW_EIAR_2009_RF.tar.gz')

duration = time.time() - start_time
print('${:,.2f}, took {:,d} ms\n'.format(predicted_value[0], int(duration * 1000)))

$32.19, took 1,505 ms

